In [1]:
import pandas as pd

In [2]:
# Read price and dividends data into dfs and parse date columns as pandas datetime dtypes
spy_price_data = pd.read_csv(r'..\1_raw_data\SPY_price_history.csv', parse_dates = ['Date'])
spy_dividend_data = pd.read_csv(r'..\1_raw_data\SPY_dividends_history.csv', parse_dates = ['Pay Date'])

In [3]:
# Left join dfs on Date and Pay Date
merged_raw_data = spy_price_data.merge(spy_dividend_data, how='left', left_on=['Date'], right_on = ['Pay Date'])

In [4]:
# Drop rows in df that contain leap days of February 29. If we don't these will cause errors in the get_yield function below
merged_raw_data = merged_raw_data[~((merged_raw_data['Date'].dt.month == 2) & (merged_raw_data['Date'].dt.day ==29))]

In [5]:
# Convert 'Date' col from pandas datetime type to just a simple date using <dt.date> method
merged_raw_data['Date'] = pd.to_datetime(merged_raw_data['Date']).dt.date

In [12]:
## ABOUT ## 
# The below code calculates the return from investing 500$ every 30 days for 5 years
# There are 7330 5-year periods from the start to end of our data
# We should try using the .apply approach as well.  Maybe Pandas is smart enough to vectorize an apply function and it may be cleaner
# We could also try using pandas.query.  It lets us write something closer to SQL when filtering the dataframes

import datetime
import pandas as pd

# Configurable
investment_amount = 500
inj_freq_days = 30
horizon_years = 5
# Date variables
inj_freq = datetime.timedelta(inj_freq_days)
start_dates = merged_raw_data['Date']
earliest_start = start_dates.min()
end_dates = start_dates + datetime.timedelta(days = 365*horizon_years)
periods = list(zip(start_dates,end_dates))
# Price at end
# NOTE: iloc handles duplicate dates which as of 5/14/2022 are still present in the dataset
todays_date = merged_raw_data['Date'].max()
todays_price = merged_raw_data[merged_raw_data['Date'] == todays_date]['Close'].iloc[0]
# Create batches of investment dates where each batch has rows {inj_freq_days} apart 
merged_raw_data['inj_group'] = ((merged_raw_data['Date'] - earliest_start) % inj_freq).dt.days
totals = []
for start_date, end_date in periods:
    # Subset {merged_raw_data} to only days that are = earliest_start + n*{inj_freq_days}
    invest_index = ((start_date - earliest_start)% inj_freq)
    invest_index = invest_index.days
    df = merged_raw_data[merged_raw_data['inj_group'] == invest_index]
    df = merged_raw_data[merged_raw_data['Date'] <= end_date]
    # Calculate {cash_at_end} for this {horizon_years} inevestment period starting at {start_date}
    stocks_owned = investment_amount/df['Close']
    todays_value = stocks_owned * todays_price
    price_at_end = df['Close'][df['Date'] == df['Date'].max()].iloc[0]
    cash_at_end = (investment_amount/df['Close'] * price_at_end).sum()
    totals.append(cash_at_end)
# There are 7330 5-year periods within our dataset. 
print(len(totals))
    

7330


In [26]:
# Write Lump Sum Investing (LSI) function "lsi_get_yield" to calculate yield of principal invested over a delta time horizon
# where the principal is invested in its entirity at the start of the delta time horizon

def lsi_get_yield(dataframe: str, date_col: str,  price_col: str, current_date: str, delta: int):
    """
    dataframe: dataframe to operate over (str)
    date_col: col name of date data (str)
    price_col: column name of price data (str)
    current_date:
    delta: investment time horizon in years (int)
    """
    current_price = dataframe.loc[dataframe[date_col] == current_date, price_col]
    current_price = current_price.reset_index()
    
    past_date = current_date - pd.Timedelta(days=delta*365)
    past_price = dataframe.loc[dataframe[date_col] == past_date, price_col]
    past_price = past_price.reset_index()

    return (current_price/past_price)[price_col].values[0]

In [27]:
# Pass lsi_get_yield function to a lambda function to the apply function to get 'Yield" column
merged_raw_data['lsi_yield'] = merged_raw_data.apply(lambda row: lsi_get_yield(dataframe=merged_raw_data, date_col='Date', current_date=row['Date'], price_col='Close', delta=5), axis=1, result_type='expand')

In [29]:
# Write Dollar Cost Averaging (DCA) function "dca_get_yield" to calculate yield of a principal invested in equal parts
# at evenly seperated intervals over the length of the delta time horizon
def dca_get_yield(dataframe: str, start_date: pd.datetime, date_col: str, price_col: str, delta: int, period_size: pd.Timedelta):
    """
    dataframe: dataframe to operate over (str)
    end_date: (str)
    price_col: (str)
    delta: investment delta in years (int)
    period_size: length of time in days between principal investments
    """
    investment_times = pd.date_range(start_date, start_date + (pd.Timedelta(days=delta*365)), period_size) # pd.date_range is like np.linspace func but for dates
    
    test_df = [dataframe.apply(lambda row: lsi_get_yield(dataframe=dataframe, date_col=date_col, current_date=row[x], price_col=price_col, delta=delta), axis=1, result_type='expand') for x in investment_times]    
    
    
    return test_df

<ipython-input-29-3dfd5a9161a3>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  def dca_get_yield(dataframe: str, start_date: pd.datetime, date_col: str, price_col: str, delta: int, period_size: pd.Timedelta):


In [ ]:
dca_get_yield(merged_raw_data, 'Date', )

In [16]:
# on to a lambda function to the apply function to get 'Yield" column
merged_raw_data['dca_yield'] = merged_raw_data.apply(lambda row: dca_get_yield(merged_raw_data, row['Date'], 'Close', 5, 14), axis=1, result_type='expand')

ValueError: Wrong number of items passed 14, placement implies 1